In [1]:
# Setup
from config import setup_env
from patterns import ModelConfig, OpenAIChat, SystemModule, PromptTemplate, Pipeline, StepContext
from typing import Final
import json
import re

# Setup environment and create model directly
setup_env()

MODEL_CONFIG : Final = ModelConfig(model_id="gpt-5", temperature=0.8, max_tokens=3000)

model = OpenAIChat(MODEL_CONFIG)

# JSON validation functions
def validate_context_json(response_text):
    """
    Validates that the response is valid JSON with the expected structure.
    Returns (is_valid, parsed_json, error_message)
    """
    try:
        # Try to extract JSON from response (in case there's extra text)
        json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
        if json_match:
            json_text = json_match.group()
        else:
            json_text = response_text.strip()
        
        # Parse JSON
        parsed = json.loads(json_text)
        
        # Validate structure
        if not isinstance(parsed, dict):
            return False, None, "Response is not a JSON object"
        
        if "contexts" not in parsed:
            return False, None, "Missing 'contexts' key in JSON"
        
        if not isinstance(parsed["contexts"], list):
            return False, None, "'contexts' must be an array"
        
        if len(parsed["contexts"]) != 5:
            return False, None, f"Expected exactly 5 contexts, got {len(parsed['contexts'])}"
        
        # Validate each context
        required_keys = ["user_role", "user_personality", "what_they_are_doing_for_current_task"]
        for i, context in enumerate(parsed["contexts"]):
            if not isinstance(context, dict):
                return False, None, f"Context {i+1} is not an object"
            
            for key in required_keys:
                if key not in context:
                    return False, None, f"Context {i+1} missing required key: {key}"
                
                if not isinstance(context[key], str) or not context[key].strip():
                    return False, None, f"Context {i+1} key '{key}' must be a non-empty string"
        
        return True, parsed, None
        
    except json.JSONDecodeError as e:
        return False, None, f"Invalid JSON format: {str(e)}"
    except Exception as e:
        return False, None, f"Validation error: {str(e)}"


✅ OpenAI API key loaded successfully


In [2]:
with open("./build_block.json", "r") as f:
    build_block = json.load(f)

block_types_list = list(build_block["agent_environment"].keys())


In [80]:
def generate_context(provided_inspiration):    
    system_prompt = """You are a creative context generator that creates diverse user personas and scenarios based on inspiration.

CRITICAL: You MUST respond with ONLY valid JSON. Do NOT include any explanatory text, markdown formatting, or additional commentary before or after the JSON. Your entire response must be parseable JSON.

Your task is to generate a JSON structure with exactly 5 diverse options, each containing:
- user_role: The role, identity, or life situation of the user
- user_personality: Key personality traits and characteristics
- what_they_are_doing_for_current_task: Specific current activity or task they're engaged in

IMPORTANT: Generate a MIX of both WORK-RELATED and CASUAL DAY-TO-DAY contexts to show the full range of possibilities.

Requirements:
1. Generate exactly 5 distinct options
2. Each option must be significantly different from the others
3. Include BOTH casual/personal contexts 
4. Vary across different settings, personality types, and task complexity
5. Make each option realistic and relatable
6. Ensure diversity in life situations

RESPONSE FORMAT - Return ONLY this exact JSON structure with NO additional text:
{
  "contexts": [
    {
      "user_role": "string",
      "user_personality": "string", 
      "what_they_are_doing_for_current_task": "string"
    }
  ]
}

Create authentic, varied personas spanning:


CASUAL DAY-TO-DAY CONTEXTS:
- Personal roles: Parent, student, hobbyist, retiree, homeowner, pet owner, etc.
- Daily activities: Household management, personal projects, hobbies, fitness, learning, etc.
- Life situations: Moving homes, planning events, organizing spaces, pursuing interests, etc.

PERSONALITY VARIETY:
- Analytical, creative, detail-oriented, big-picture, social, introverted, practical, dreamer, organized, spontaneous, etc.

REMINDER: Respond with ONLY the JSON object. No explanations, no markdown, no additional text."""

    feedback_history = []
    iteration = 1
    
    while True:
        print(f"\n{'='*60}")
        print(f"GENERATION ATTEMPT #{iteration}")
        print(f"{'='*60}")
        
        # Prepare the user message with feedback if available
        if feedback_history:
            user_message = f"""Create 5 diverse user contexts based on this inspiration: {provided_inspiration}

Previous feedback from user:
{chr(10).join(feedback_history)}

Please incorporate this feedback and generate improved contexts."""
        else:
            user_message = f"Create 5 diverse user contexts based on this inspiration: {provided_inspiration}"
        
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]

        # Retry logic for JSON validation
        max_retries = 3
        retry_count = 0
        valid_response = None
        
        while retry_count < max_retries:
            response = model.generate(messages)
            
            # Validate JSON response
            is_valid, parsed_json, error_message = validate_context_json(response)
            
            if is_valid:
                print("Generated Context Options:")
                print(json.dumps(parsed_json, indent=2))
                valid_response = response
                break
            else:
                retry_count += 1
                print(f"JSON Validation Error (Attempt {retry_count}/{max_retries}): {error_message}")
                print(f"Raw response: {response[:200]}...")
                
                if retry_count < max_retries:
                    print("Retrying with stricter JSON requirements...")
                    
                    # Add more specific JSON instruction for retry
                    retry_message = user_message + "\n\nCRITICAL: Your response must be ONLY valid JSON with no additional text, explanations, or formatting."
                    messages = [
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": retry_message}
                    ]
                else:
                    print("Max retries reached. Using raw response (may contain invalid JSON):")
                    print(response)
                    valid_response = response
        
        print(f"\n{'='*60}")
        
        # Get user feedback
        feedback = input("\nProvide feedback (or type 'good' to finish): ").strip()
        
        if feedback.lower() in ['done', 'good', 'good!', 'looks good', 'perfect']:
            print("\nGreat! Context generation completed successfully.")
            return valid_response
        
        if feedback:
            feedback_history.append(f"Iteration {iteration}: {feedback}")
            print(f"Feedback recorded: {feedback}")
            print("Generating new contexts based on your feedback...")
        else:
            print("No feedback provided. Generating new contexts...")
        
        iteration += 1


In [81]:
provided_inspiration="""
+ Implement a persona challenge where the model alternates between acting as a formal business consultant and a friendly peer, depending on the type of request or tool being used, to provide varied and contextually appropriate responses. 
+ Introduce a tool selection challenge by instructing the model to prioritize tools that provide direct user feedback over those that do not, ensuring a more interactive and transparent experience, especially when gathering system information. 
+ Enforce a verification challenge where the model must seek explicit confirmation from the user before making any changes to critical settings, such as connectivity or location services, to prevent unintended disruptions or security issues."""
context = generate_context(provided_inspiration)



GENERATION ATTEMPT #1
Generated Context Options:
{
  "contexts": [
    {
      "user_role": "UX lead at a digital health startup",
      "user_personality": "analytical, empathetic, systems-thinker, organized",
      "what_they_are_doing_for_current_task": "Drafting conversation rules so the assistant acts as a formal business consultant for billing and insurance queries and a friendly peer for wellness tips; selecting monitoring tools that show user-visible status and feedback; enforcing explicit user confirmation before any connectivity or location setting changes in the app."
    },
    {
      "user_role": "First-year university student majoring in computer science",
      "user_personality": "curious, experimental, budget-conscious, slightly introverted",
      "what_they_are_doing_for_current_task": "Configuring a personal study-buddy chatbot to switch tone between a formal tutor for problem sets and a casual classmate for motivation; choosing browser tools that provide immediat

KeyboardInterrupt: 

In [83]:
context = """    {
      "user_role": "Busy parent coordinating a multi-device smart home",
      "user_personality": "practical, nurturing, schedule-driven, frugal",
      "what_they_are_doing_for_current_task": "Setting up the home assistant to speak like a formal advisor for budgeting and energy reports and like a friendly peer for family reminders; choosing apps that show clear prompts and feedback; ensuring the assistant asks for confirmation before changing router settings, geofencing, or parental controls."
    },"""


In [48]:
def validate_structured_response(response, provided_inspiration, model):
    """
    Validate the structured response from generate_block function.
    
    Args:
        response: Generated response to validate
        provided_inspiration: Original inspiration text to check against
        model: OpenAI model for content validation
    
    Returns:
        tuple: (is_valid, list_of_errors)
    """
    errors = []
    
    # Check 1: All 5 building block types mentioned
    required_blocks = [
        "[CONTEXT_INFORMATION]",
        "[TOOL_USE_INSTRUCTIONS]", 
        "[USER_PREFERENCES]",
        "[BACKGROUND_INFORMATION]",
        "[TONAL_CONTROL]"
    ]
    
    missing_blocks = []
    for block in required_blocks:
        if block not in response:
            missing_blocks.append(block)
    
    if missing_blocks:
        errors.append(f"Missing building blocks: {', '.join(missing_blocks)}")
    
    # Check 2: Number of paragraphs (6-8)
    # Count paragraphs by splitting on double newlines and filtering non-empty
    paragraphs = [p.strip() for p in response.split('\n\n') if p.strip()]
    # Filter out "You are" if it's standalone
    paragraphs = [p for p in paragraphs if p.strip().lower() != "you are"]
    
    paragraph_count = len(paragraphs)
    if paragraph_count < 6 or paragraph_count > 8:
        errors.append(f"Wrong number of paragraphs: {paragraph_count} (should be 6-8)")
    
    # Check 3: ChatGPT validation of inspiration content
    if provided_inspiration and provided_inspiration.strip():
        validation_prompt = f"""Check if this generated response contains all the key ideas from the provided inspiration.

Generated Response:
{response}

Original Inspiration:
{provided_inspiration}

Answer with just "YES" if all inspiration ideas are incorporated (using different wording is fine), or "NO" followed by what's missing."""
        
        try:
            validation_messages = [
                {"role": "user", "content": validation_prompt}
            ]
            validation_response = model.generate(validation_messages)
            
            if not validation_response.strip().upper().startswith("YES"):
                errors.append(f"Inspiration content validation failed: {validation_response}")
        except Exception as e:
            errors.append(f"Could not validate inspiration content: {str(e)}")
    
    is_valid = len(errors) == 0
    return is_valid, errors


In [72]:
def generate_block( provided_inspiration):
    """
    Generate a 6-8 paragraph system prompt using building block format with interactive feedback.
    
    Args:
        context: String containing context information
        provided_inspiration: String containing ideas to incorporate
    
    Returns:
        String: The generated structured system prompt
    """
    
    max_iterations = 5
    iteration = 1
    feedback_history = []
    
    with open("./build_block.json", "r") as f:
        build_block = f.read()
    
    # Base system prompt for generating structured prompts
    system_prompt = f"""You are a system prompt generator that creates structured prompts using building block format.

Create 6 to 8 paragraphs using building block format.

Format Structure:
Each paragraph should follow patterns like:
- [block name] [block name] (content that paraphrases ideas) [block name]
- [block name] (content with paraphrased ideas) [block name] [block name]
- (content incorporating paraphrased ideas) [block name] [block name] [block name]
- [block name] [block name] [block name]
- [block name] [block name] 


The order of block names is completely flexible - you can place them at the beginning, middle, end, or mixed throughout each paragraph as makes sense for natural flow.

Requirements:
1. ONLY SHOW AS BLOCK NAME and (paraphrased ideas) in the paragraph, do not have any other text in the paragraph.
2. Create 6 to 8 paragraphs (choose the number that works best for the content)
3. Each paragraph uses 2-3 building block references. At least 1 paragraph only has [block name] [block name].
4. Adapt everything to be suitable for the given context
5. This is to generate a system prompt, so structure it appropriately
6. Use ONLY these specific building block names:
   - [CONTEXT_INFORMATION]
   - [TOOL_USE_INSTRUCTIONS] 
   - [USER_PREFERENCES]
   - [BACKGROUND_INFORMATION]
   - [TONAL_CONTROL]
7. Reference this as example {build_block}
8. (text inside round brackets) is paraphrased from inspiration ideas only and should be a full paraphase, and can easily identify as a paraphrased version of the inspiration ideas. Should have 1 to 1 map between paraphase and inspiration ideas.
9. THERE MUST BE AT LEAST 2 PARAGRAPHS that is [block name] [block name] [block name] only without paraphrased ideas.


Generate the system prompt using the flexible building block format"""
    
    # Create model with 2500 token limit
    structured_model_config = ModelConfig(model_id="gpt-5", temperature=0.7, max_tokens=2500)
    structured_model = OpenAIChat(structured_model_config)
    
    while iteration <= max_iterations:
        print(f"\n{'='*60}")
        print(f"GENERATING STRUCTURED PROMPT - Iteration {iteration}")
        print(f"{'='*60}")
        
        # Build user message
        user_message = ""
        
        
        if provided_inspiration:
            user_message += f"""

Inspiration ideas to incorporate throughout the paragraphs is in bullet points.
{provided_inspiration}

Transform these ideas using different wording and distribute them naturally across the paragraphs. They can appear at any position within each paragraph.
"""
        
        # Add feedback history if exists
        if feedback_history:
            user_message += "\n\nPrevious feedback to incorporate:\n" + "\n".join(feedback_history)
        
        # Automatic retry with validation
        max_retries = 3
        retry_count = 0
        valid_response = None
        
        while retry_count < max_retries:
            try:
                # Generate response
                messages = [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_message.strip()}
                ]
                
                print(f"Generating structured prompt... (Attempt {retry_count + 1}/{max_retries})")
                response = structured_model.generate(messages)
                
                # Validation checks
                validation_passed, validation_errors = validate_structured_response(response, provided_inspiration, structured_model)
                
                if validation_passed:
                    print("✅ Validation passed!")
                    print("Generated Structured Prompt:")
                    print("-" * 40)
                    print(response)
                    print("-" * 40)
                    valid_response = response
                    break
                else:
                    retry_count += 1
                    print(f"❌ Validation failed (Attempt {retry_count}/{max_retries}):")
                    for error in validation_errors:
                        print(f"  - {error}")
                    
                    if retry_count < max_retries:
                        print("Retrying with validation feedback...")
                        # Add validation feedback to user message for next attempt
                        validation_feedback = "Previous attempt failed validation:\n" + "\n".join(validation_errors)
                        user_message += f"\n\nIMPORTANT - Fix these issues:\n{validation_feedback}"
                    else:
                        print("Max retries reached. Using last response despite validation issues.")
                        print("Generated Structured Prompt (with validation issues):")
                        print("-" * 40)
                        print(response)
                        print("-" * 40)
                        valid_response = response
                
            except Exception as e:
                retry_count += 1
                print(f"Error generating structured prompt (Attempt {retry_count}/{max_retries}): {str(e)}")
                if retry_count >= max_retries:
                    valid_response = f"Error: {str(e)}"
        
        response = valid_response
        
        print(f"\n{'='*60}")
        
        # Get user feedback
        feedback = input("\nProvide feedback (or type 'good' to finish, 'stop' to end): ").strip()

        if feedback == "":
            print("No feedback provided. Generating new structured prompt...")
            return response
        
        if feedback.lower() in ['done', 'good', 'good!', 'looks good', 'perfect']:
            print("\nGreat! Structured prompt generation completed successfully.")
            return response
            
        if feedback.lower() in ['stop', 'quit', 'exit']:
            print("\nStopping structured prompt generation.")
            return response
        
        if feedback:
            feedback_history.append(f"Iteration {iteration}: {feedback}")
            print(f"Feedback recorded: {feedback}")
            print("Generating new structured prompt based on your feedback...")
        else:
            print("No feedback provided. Generating new structured prompt...")
        
        iteration += 1
    
    print(f"\nReached maximum iterations ({max_iterations}). Returning final result.")
    return response 


In [85]:
# block_structure = generate_block(provided_inspiration)
complex_structure = generate_complex_block(block_structure, context)
system_settings = '''[
  {
    "cellular": true,
    "device_id": "30598b7d-eba3-4801-ace0-270e0cd18a83",
    "formatted_address": "Lee Hall Annex, 1000 W Garden Ave, Coeur d'Alene, ID 83814, USA",
    "latitude": 47.6763288,
    "locale": "en_US",
    "location_service": true,
    "longitude": -116.7996314,
    "low_battery_mode": true,
    "place_id": "ChlJ_4gp6oXGYVMRFtEFeulbQwg",
    "utc_offset_seconds": -25200,
    "wifi": false
  }
]'''
complex_structure = add_system_info(complex_structure, context, system_settings)




ADDING COMPLEX BLOCKS - Iteration 1
Adding complex blocks... (Attempt 1/3)
✅ Validation passed!
Generated Requirements Structure:
----------------------------------------
(Switch personas based on task and tool: adopt a polished consultant voice for business-like requests, and shift to a warm, collegial style for peer-level queries, so replies match the situation.) [TONAL_CONTROL#Define_Personality_and_Tone] [USER_PREFERENCES] #Implement Dynamic Behavior Scaling# (adjust tone and verbosity based on task complexity and stakes) [CONTEXT_INFORMATION] #Provide Context Information about Applications and Entities# (detect active apps or channels to decide which persona to use)

[TOOL_USE_INSTRUCTIONS#Guide_Tool_Use_and_Response_Formatting] (Prefer utilities that surface immediate feedback to the user over silent ones, particularly when inspecting device or system details, to keep the process interactive and clear.) [CONTEXT_INFORMATION] #Provide Context Information about Applications and En

In [87]:
len('''You are assisting a busy parent who coordinates a multi-device smart home. The parent is practical, nurturing, schedule-driven, and frugal.

Use a formal advisor voice for budgeting, energy reports, or household planning. Use a friendly peer voice for reminders, schedules, or casual device help. Keep one tone per reply, without mixing.

Adjust detail to task complexity. Keep answers short and warm for simple tasks. Provide clear, structured, step-by-step analysis for complex or high-impact tasks such as budgeting or energy optimization.

System state: cellular data enabled, locale en_US, location services active, power mode low battery, Wi-Fi inactive. Choose apps or utilities that provide visible prompts and live status. If multiple tools can do the same task, prefer the one that shows clear progress.

Always request explicit confirmation before making sensitive changes such as router settings, geofencing, parental controls, or network access. Verify commands and intent before executing. Respect household rules and constraints as non-negotiable.

Maintain clarity, neutrality, and practical focus. Provide actionable information with only relevant details. Always prioritize tools and methods that give visible confirmation so the parent can confidently approve or reject actions.''')


1297

In [58]:
def validate_requirements_response(response, model):
    """
    Validate the requirements structure response.
    Checks paragraphs, format, building blocks, and complex block coverage.
    """
    errors = []
    
    # Load complex blocks from JSON to check coverage
    with open("./complex_block.json", "r") as f:
        complex_blocks = json.load(f)
    
    required_complex_blocks = list(complex_blocks.keys())
    
    # Check 1: Number of paragraphs (6-8)
    paragraphs = [p.strip() for p in response.split('\n\n') if p.strip()]
    paragraphs = [p for p in paragraphs if p.strip().lower() != "you are"]
    
    paragraph_count = len(paragraphs)
    if paragraph_count < 6 or paragraph_count > 8:
        errors.append(f"Wrong number of paragraphs: {paragraph_count} (should be 6-8)")
    
    # Check 2: Format validation - should have mixed format (both #block# and merged [BLOCK#complex])
    has_separate_format = bool(re.search(r'#[^#]+#', response))
    has_merged_format = bool(re.search(r'\[[A-Z_]+#[A-Za-z_\s]+\]', response))
    
    if not has_separate_format and not has_merged_format:
        errors.append("Missing both separate #block# and merged [BLOCK#complex] formats")
    elif not has_separate_format:
        errors.append("Missing separate #block# format")
    elif not has_merged_format:
        errors.append("Missing merged [BLOCK#complex] format")
    
    # Check 3: Should still contain building blocks (either standalone or merged)
    if not re.search(r'\[[A-Z_]+\]', response) and not re.search(r'\[[A-Z_]+#', response):
        errors.append("Missing building block format [BLOCK_NAME]")
    
    # Check 4: Complex block coverage - Python check for ALL types included (MUST HAVE ALL)
    # Handle both separate format (#block#) and merged format ([BLOCK#complex])
    missing_complex_blocks = []
    for complex_block_name in required_complex_blocks:
        # Check both formats: separate #block# and merged [BLOCK#complex]
        short_name = complex_block_name.replace(" ", "_").replace(",", "").replace("-", "_")
        found_separate = f"#{complex_block_name}#" in response
        found_merged = (f"#{complex_block_name}" in response or 
                       f"#{short_name}" in response or
                       complex_block_name.replace(" ", "_") in response)
        
        if not found_separate and not found_merged:
            missing_complex_blocks.append(complex_block_name)
    
    # STRICT REQUIREMENT: ALL 7 complex blocks must be included
    if missing_complex_blocks:
        errors.append(f"Missing required complex blocks ({len(missing_complex_blocks)}/7 missing): {', '.join(missing_complex_blocks)}")
    
    # Check 5: Verify we have exactly all 7 complex blocks
    found_complex_blocks = []
    for complex_block_name in required_complex_blocks:
        short_name = complex_block_name.replace(" ", "_").replace(",", "").replace("-", "_")
        found_separate = f"#{complex_block_name}#" in response
        found_merged = (f"#{complex_block_name}" in response or 
                       f"#{short_name}" in response or
                       complex_block_name.replace(" ", "_") in response)
        
        if found_separate or found_merged:
            found_complex_blocks.append(complex_block_name)
    
    if len(found_complex_blocks) != 7:
        errors.append(f"Incomplete complex block coverage: {len(found_complex_blocks)}/7 complex blocks found (ALL 7 REQUIRED)")
    
    # Check 6: At least 3 paragraphs must have 2 different complex blocks (handle both formats)
    paragraphs_with_multiple_blocks = 0
    for paragraph in paragraphs:
        # Count unique complex blocks in this paragraph (both separate and merged formats)
        blocks_in_paragraph = set()
        for complex_block_name in required_complex_blocks:
            short_name = complex_block_name.replace(" ", "_").replace(",", "").replace("-", "_")
            
            # Check separate format #block#
            if f"#{complex_block_name}#" in paragraph:
                blocks_in_paragraph.add(complex_block_name)
            
            # Check merged format [BLOCK#complex]
            elif (f"#{complex_block_name}" in paragraph or 
                  f"#{short_name}" in paragraph or
                  complex_block_name.replace(" ", "_") in paragraph):
                blocks_in_paragraph.add(complex_block_name)
        
        # Check if this paragraph has 2 or more different complex blocks
        if len(blocks_in_paragraph) >= 2:
            paragraphs_with_multiple_blocks += 1
    
    if paragraphs_with_multiple_blocks < 3:
        errors.append(f"Insufficient paragraph complexity: only {paragraphs_with_multiple_blocks} paragraphs have 2+ complex blocks (need at least 3)")
    
    is_valid = len(errors) == 0
    return is_valid, errors


In [59]:
def generate_complex_block(block_output, context=None):
    """
    Simplified: Add complex block identifiers from complex_block.json to existing building blocks.
    Format: #Block Name# (short explanation)
    Uses definitions and examples from the JSON file for accurate implementation.
    """
    max_iterations = 5
    iteration = 1
    feedback_history = []
    
    # Load complex blocks from JSON with definitions and examples
    with open("./complex_block.json", "r") as f:
        complex_blocks = json.load(f)
    
    # Create detailed information about each complex block
    complex_block_info = ""
    for block_name, block_data in complex_blocks.items():
        complex_block_info += f"\n- {block_name}:\n"
        complex_block_info += f"  Definition: {block_data['Definition']}\n"
        complex_block_info += f"  Examples: {'; '.join(block_data['Examples'])}\n"
    
    system_prompt = f"""You are a system prompt processor that adds complex block identifiers to building block structures using MIXED FORMAT.

Your task:
1. Keep the existing paragraph structure EXACTLY as is
2. Add ALL 7 complex block identifiers based on their definitions - EVERY SINGLE ONE MUST BE INCLUDED
3. Use MIXED FORMAT: 30-40% merged format, 60-70% separate format
4. Use the provided definitions and examples to ensure correct application

MIXED FORMAT REQUIREMENTS:
- MERGED FORMAT (30-40% of building blocks): [BUILDING_BLOCK#Complex_Block_Name]
- SEPARATE FORMAT (60-70% of building blocks): [BUILDING_BLOCK] #Complex Block Name# (explanation)

CRITICAL REQUIREMENT: ALL 7 complex blocks must be included in the output. No exceptions.

Available complex blocks with definitions and examples (ALL MUST BE USED):
{complex_block_info}

Instructions:
- Keep EXACT same paragraph structure and content
- Use MIXED FORMAT: Some blocks merged [BLOCK#complex], others separate [BLOCK] #complex#
- Aim for 30-40% of building blocks to use merged format [BUILDING_BLOCK#Complex_Name]
- Remaining 60-70% use separate format [BUILDING_BLOCK] #Complex Name# (brief explanation)
- MANDATORY: Include ALL 7 complex blocks - distribute them across the paragraphs appropriately
- MANDATORY: At least 3 paragraphs must have 2 different complex blocks
- Maintain all original content including (parenthetical content)

EXAMPLES:
- Merged: [BACKGROUND_INFORMATION#Define_Personality_and_Tone] (explanation)
- Separate: [TONAL_CONTROL] #Guide Tool Use and Response Formatting# (control formatting) 
- Mixed paragraph: [CONTEXT_INFORMATION#Provide_Context_Information] [USER_PREFERENCES] #Set Clear Guardrails# (safety)

VALIDATION: The output will be checked to ensure:
- ALL 7 complex blocks are present
- At least 3 paragraphs have 2 different complex blocks
- Mixed format used (30-40% merged, 60-70% separate)
Missing any requirement will cause validation failure.

Process the building block structure now and ensure ALL 7 complex blocks are included with proper mixed format distribution."""
    
    structured_model_config = ModelConfig(model_id="gpt-5", temperature=0.7, max_tokens=2500)
    structured_model = OpenAIChat(structured_model_config)
    
    while iteration <= max_iterations:
        print(f"\n{'='*60}")
        print(f"ADDING COMPLEX BLOCKS - Iteration {iteration}")
        print(f"{'='*60}")
        
        user_message = f"""
Add relevant complex block identifiers to this building block structure:

{block_output}

Context: {context if context else "General use"}
"""
        
        if feedback_history:
            user_message += "\n\nPrevious feedback to incorporate:\n" + "\n".join(feedback_history)
        
        # Automatic retry with validation
        max_retries = 3
        retry_count = 0
        valid_response = None
        
        while retry_count < max_retries:
            try:
                messages = [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_message.strip()}
                ]
                
                print(f"Adding complex blocks... (Attempt {retry_count + 1}/{max_retries})")
                response = structured_model.generate(messages)
                
                validation_passed, validation_errors = validate_requirements_response(response, structured_model)
                
                if validation_passed:
                    print("✅ Validation passed!")
                    print("Generated Requirements Structure:")
                    print("-" * 40)
                    print(response)
                    print("-" * 40)
                    valid_response = response
                    break
                else:
                    retry_count += 1
                    print(f"❌ Validation failed (Attempt {retry_count}/{max_retries}):")
                    for error in validation_errors:
                        print(f"  - {error}")
                    
                    if retry_count < max_retries:
                        validation_feedback = "Previous attempt failed validation:\n" + "\n".join(validation_errors)
                        user_message += f"\n\nIMPORTANT - Fix these issues:\n{validation_feedback}"
                    else:
                        print("Max retries reached. Using last response despite validation issues.")
                        valid_response = response
                
            except Exception as e:
                retry_count += 1
                print(f"Error (Attempt {retry_count}/{max_retries}): {str(e)}")
                if retry_count >= max_retries:
                    valid_response = f"Error: {str(e)}"
        
        response = valid_response
        print(f"\n{'='*60}")
        
        feedback = input("\nProvide feedback (or type 'good' to finish, 'stop' to end): ").strip()
        
        if feedback == "":
            return response
        
        if feedback.lower() in ['done', 'good', 'good!', 'looks good', 'perfect']:
            print("\nGreat! Requirements structure processing completed successfully.")
            return response
            
        if feedback.lower() in ['stop', 'quit', 'exit']:
            return response
        
        if feedback:
            feedback_history.append(f"Iteration {iteration}: {feedback}")
            print(f"Feedback recorded: {feedback}")
        
        iteration += 1
    
    return response


In [ ]:
def populate_content_from_complex_block_improved(complex_block, context=None):
    """
    Generate actual system prompt content from complex block identifiers.
    Uses both structure.json and complex_block.json for accurate content generation.
    
    Args:
        complex_block: String containing complex block identifiers like #Block Name# 
        context: Optional context information to tailor the content
    
    Returns:
        String: The populated system prompt with actual content
    """
    
    # Load both structure definitions and complex block definitions
    with open("./structure.json", "r") as f:
        requirement_structure_explain = json.load(f)['requirements']
    
    with open("./complex_block.json", "r") as f:
        complex_blocks = json.load(f)
    
    max_iterations = 5
    iteration = 1
    feedback_history = []
    
    # Extract context information for tailoring
    context_info = ""
    if context:
        context_info = f"Context: {context}"
    
    # Create detailed complex block information
    complex_block_info = ""
    for block_name, block_data in complex_blocks.items():
        complex_block_info += f"\n- {block_name}:\n"
        complex_block_info += f"  Definition: {block_data['Definition']}\n"
        complex_block_info += f"  Examples: {'; '.join(block_data['Examples'])}\n"
    
    # System prompt for generating actual content
    system_prompt = f"""You are a system prompt content generator that converts complex block identifiers into actual system prompt content.

Your task:
1. Replace each complex block identifier (like #Block Name#) with actual system prompt content
2. Use the provided definitions and examples from complex_block.json for each block
3. Use structure.json for additional technical details if needed
4. Tailor the content to be suitable for the given context
5. Follow the format: "actual_content [complexity_block]" where complexity_block is the original identifier
6. Generate natural, coherent system prompt content that flows well

Available complex block definitions and examples:
{complex_block_info}

{context_info}

Instructions:
- Keep the exact same paragraph structure as the input
- Replace each #identifier# with appropriate content based on its definition followed by [complexity_block]
- Content should be natural system prompt instructions that implement the definition
- Use the examples provided to understand how to apply each block correctly
- Tailor examples and language to the provided context
- Maintain coherent flow between sentences
- Each piece of content should be actionable system prompt instructions

Process the provided complex block structure now."""
    
    # Create model for content generation
    content_model_config = ModelConfig(model_id="gpt-4", temperature=0.7, max_tokens=3000)
    content_model = OpenAIChat(content_model_config)
    
    while iteration <= max_iterations:
        print(f"\n{'='*60}")
        print(f"POPULATING CONTENT FROM COMPLEX BLOCK - Iteration {iteration}")
        print(f"{'='*60}")
        
        # Build user message with the complex block
        user_message = f"""
Please convert each complex block identifier into actual system prompt content while maintaining the exact same structure.

Here is the complex block structure to populate:

{complex_block}

Remember to:
- Replace each #identifier# with relevant content based on its definition + [complexity_block]
- Keep all parenthetical content unchanged
- Use the definitions and examples from complex_block.json to ensure accuracy
- Tailor content to the provided context
- Make content actionable and clear
- Maintain natural flow
"""
        
        # Add feedback history if exists
        if feedback_history:
            user_message += "\n\nPrevious feedback to incorporate:\n" + "\n".join(feedback_history)
        
        try:
            # Generate response
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_message.strip()}
            ]
            
            print("Populating complex block with actual content...")
            response = content_model.generate(messages)
            
            print("Generated System Prompt Content:")
            print("-" * 40)
            print(response)
            print("-" * 40)
            
        except Exception as e:
            print(f"Error populating content: {str(e)}")
            response = f"Error: {str(e)}"
        
        print(f"\n{'='*60}")
        
        # Get user feedback
        feedback = input("\nProvide feedback (or type 'good' to finish, 'stop' to end): ").strip()
        
        if feedback == "":
            print("No feedback provided. Content population complete...")
            return response
        
        if feedback.lower() in ['done', 'good', 'good!', 'looks good', 'perfect']:
            print("\nGreat! Content population completed successfully.")
            return response
            
        if feedback.lower() in ['stop', 'quit', 'exit']:
            print("\nStopping content population.")
            return response
        
        if feedback:
            feedback_history.append(f"Iteration {iteration}: {feedback}")
            print(f"Feedback recorded: {feedback}")
            print("Re-generating with your feedback...")
        else:
            print("No feedback provided. Re-generating...")
        
        iteration += 1
    
    print(f"\nReached maximum iterations ({max_iterations}). Returning final result.")
    return response


In [ ]:
def analyze_complex_block_coverage(response):
    """
    Analyze and display complex block coverage in the response.
    Shows which blocks are included and which are missing.
    """
    # Load complex blocks from JSON
    with open("./complex_block.json", "r") as f:
        complex_blocks = json.load(f)
    
    all_complex_blocks = list(complex_blocks.keys())
    found_blocks = []
    missing_blocks = []
    
    print("=== COMPLEX BLOCK COVERAGE ANALYSIS ===")
    print(f"Total available complex blocks: {len(all_complex_blocks)}")
    print()
    
    for block_name in all_complex_blocks:
        if f"#{block_name}#" in response:
            found_blocks.append(block_name)
            print(f"✅ FOUND: {block_name}")
        else:
            missing_blocks.append(block_name)
            print(f"❌ MISSING: {block_name}")
    
    print()
    print(f"Coverage Summary: {len(found_blocks)}/{len(all_complex_blocks)} complex blocks included")
    print(f"Coverage Percentage: {(len(found_blocks)/len(all_complex_blocks))*100:.1f}%")
    
    if missing_blocks:
        print(f"\nMissing blocks ({len(missing_blocks)}):")
        for block in missing_blocks:
            print(f"  - {block}")
            print(f"    Definition: {complex_blocks[block]['Definition'][:100]}...")
    
    return len(found_blocks), len(missing_blocks)

# Test function to show all available complex blocks
def show_all_complex_blocks():
    """Display all available complex blocks with their definitions."""
    with open("./complex_block.json", "r") as f:
        complex_blocks = json.load(f)
    
    print("=== ALL AVAILABLE COMPLEX BLOCKS ===")
    for i, (block_name, block_data) in enumerate(complex_blocks.items(), 1):
        print(f"{i}. {block_name}")
        print(f"   Definition: {block_data['Definition']}")
        print(f"   Examples: {len(block_data['Examples'])} provided")
        print()

# Show all available blocks
show_all_complex_blocks()


In [45]:
def populate_content_from_complex_block(complex_block, context=None):
    """
    Generate actual system prompt content from complex block identifiers.
    
    Args:
        complex_block: String containing complex block identifiers like [background_information@dynamic_behavior_scaling@complexity_assessment]
        context: Optional context information to tailor the content
    
    Returns:
        String: The populated system prompt with actual content
    """
    
    with open("./structure.json", "r") as f:
        requirement_structure_explain = json.load(f)['requirements']
    
    max_iterations = 5
    iteration = 1
    feedback_history = []
    
    # Extract context information for tailoring
    context_info = ""
    if context:
        context_info = f"Context: {context}"
    
    # System prompt for generating actual content
    system_prompt = f"""You are a system prompt content generator that converts complex block identifiers into actual system prompt content.

Your task:
1. Replace each complex block identifier (like [background_information@dynamic_behavior_scaling@complexity_assessment]) with actual system prompt content
2. Use the provided explanations and examples from the requirement structure
3. Tailor the content to be suitable for the given context
4. Follow the format: "actual_content [complexity_block]" where complexity_block is the original identifier
5. Generate natural, coherent system prompt content that flows well

Available requirement explanations and examples:
{json.dumps(requirement_structure_explain, indent=2)}

{context_info}

Instructions:
- Keep the exact same paragraph structure as the input
- Replace each [identifier] with appropriate content followed by [complexity_block]
- Content should be natural system prompt instructions
- Tailor examples and language to the provided context
- Maintain coherent flow between sentences
- Each piece of content should be actionable system prompt instructions

Process the provided complex block structure now."""
    
    # Create model for content generation
    content_model_config = ModelConfig(model_id="gpt-5", temperature=0.7, max_tokens=3000)
    content_model = OpenAIChat(content_model_config)
    
    while iteration <= max_iterations:
        print(f"\n{'='*60}")
        print(f"POPULATING CONTENT FROM COMPLEX BLOCK - Iteration {iteration}")
        print(f"{'='*60}")
        
        # Build user message with the complex block
        user_message = f"""
Please convert each complex block identifier into actual system prompt content while maintaining the exact same structure.

Here is the complex block structure to populate:

{complex_block}

Remember to:
- Replace each [identifier] with relevant content + [complexity_block]
- Keep all parenthetical content unchanged
- Tailor content to the context
- Make content actionable and clear
- Maintain natural flow
"""
        
        # Add feedback history if exists
        if feedback_history:
            user_message += "\n\nPrevious feedback to incorporate:\n" + "\n".join(feedback_history)
        
        try:
            # Generate response
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_message.strip()}
            ]
            
            print("Populating complex block with actual content...")
            response = content_model.generate(messages)
            
            print("Generated System Prompt Content:")
            print("-" * 40)
            print(response)
            print("-" * 40)
            
        except Exception as e:
            print(f"Error populating content: {str(e)}")
            response = f"Error: {str(e)}"
        
        print(f"\n{'='*60}")
        
        # Get user feedback
        feedback = input("\nProvide feedback (or type 'good' to finish, 'stop' to end): ").strip()
        
        if feedback == "":
            print("No feedback provided. Content population complete...")
            return response
        
        if feedback.lower() in ['done', 'good', 'good!', 'looks good', 'perfect']:
            print("\nGreat! Content population completed successfully.")
            return response
            
        if feedback.lower() in ['stop', 'quit', 'exit']:
            print("\nStopping content population.")
            return response
        
        if feedback:
            feedback_history.append(f"Iteration {iteration}: {feedback}")
            print(f"Feedback recorded: {feedback}")
            print("Re-generating with your feedback...")
        else:
            print("No feedback provided. Re-generating...")
        
        iteration += 1
    
    print(f"\nReached maximum iterations ({max_iterations}). Returning final result.")
    return response


In [63]:
def add_system_info(complex_structure, context, system_settings):
    """
    Add 2-5 pieces of system setting information to the FIRST CONTEXT_INFORMATION block.
    
    Args:
        complex_structure: String containing the complex structure (JSON format as string)
        context: String containing user context information
        system_settings: String containing system-specific settings and configurations
    
    Returns:
        String: Enhanced complex structure with system info added to first CONTEXT_INFORMATION block
    """
    
    max_iterations = 3
    iteration = 1
    feedback_history = []
    
    system_prompt = """You are a system prompt enhancer that adds system setting information to the FIRST CONTEXT_INFORMATION block.

Your task:
1. Analyze the provided context and system settings to understand the user's environment
2. Generate 2-5 pieces of relevant system setting information
3. Add this information ONLY to the FIRST CONTEXT_INFORMATION block you find in the structure
4. Leave all other blocks completely unchanged

Requirements:
- Add 2-5 pieces of system setting information to the FIRST CONTEXT_INFORMATION block only
- Generate system info dynamically based on context and settings
- Maintain the exact structure and format of the input
- Make additions feel natural and integrated
- Do NOT modify any other blocks

Instructions:
- Find the FIRST CONTEXT_INFORMATION block in the structure
- Add 2-5 pieces of relevant system setting information to that block only
- Generate system info that would be helpful for the specific context and settings
- Format: Add as natural extensions within the first CONTEXT_INFORMATION block
- Example: "[CONTEXT_INFORMATION] existing content (system: mobile device, storage: 64GB, network: WiFi)"

CRITICAL: Enhance ONLY the FIRST CONTEXT_INFORMATION block. Leave all others unchanged.

Focus on adding system setting information that's specifically relevant to the context."""
    
    model_config = ModelConfig(model_id="gpt-4", temperature=0.7, max_tokens=3000)
    model = OpenAIChat(model_config)
    
    while iteration <= max_iterations:
        print(f"\n{'='*50}")
        print(f"ADDING SYSTEM INFO TO FIRST CONTEXT_INFORMATION BLOCK - Iteration {iteration}")
        print(f"{'='*50}")
        
        user_message = f"""
Add 2-5 pieces of system setting information to the FIRST CONTEXT_INFORMATION block:

COMPLEX STRUCTURE:
{complex_structure}

CONTEXT:
{context}

SYSTEM SETTINGS:
{system_settings}

Requirements:
- Add 2-5 pieces of system setting information to the FIRST CONTEXT_INFORMATION block only
- Generate relevant system info based on context and system settings
- Maintain exact structure and format
- Leave all other blocks unchanged
"""
        
        if feedback_history:
            user_message += "\n\nPrevious feedback to incorporate:\n" + "\n".join(feedback_history)
        
        try:
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_message.strip()}
            ]
            
            print("Adding system setting information to first CONTEXT_INFORMATION block...")
            response = model.generate(messages)
            
            print("Enhanced Complex Structure:")
            print("-" * 40)
            print(response)
            print("-" * 40)
                
        except Exception as e:
            print(f"Error adding system info: {str(e)}")
            response = f"Error: {str(e)}"
        
        print(f"\n{'='*50}")
        
        # Get user feedback
        feedback = input("\nProvide feedback (or type 'good' to finish, 'stop' to end): ").strip()
        
        if feedback == "":
            print("No feedback provided. System info addition complete...")
            return response
        
        if feedback.lower() in ['done', 'good', 'good!', 'looks good', 'perfect']:
            print("\nGreat! System info addition completed successfully.")
            return response
            
        if feedback.lower() in ['stop', 'quit', 'exit']:
            print("\nStopping system info addition.")
            return response
        
        if feedback:
            feedback_history.append(f"Iteration {iteration}: {feedback}")
            print(f"Feedback recorded: {feedback}")
            print("Re-generating with your feedback...")
        else:
            print("No feedback provided. Re-generating...")
        
        iteration += 1
    
    print(f"\nReached maximum iterations ({max_iterations}). Returning final result.")
    return response
